# American Sign Language - Computer Vision Project

- Dataset: https://public.roboflow.com/object-detection/american-sign-language-letters
- Example Task: https://towardsdatascience.com/sign-language-recognition-with-advanced-computer-vision-7b74f20f3442

In [ ]:
import numpy as np
import tensorflow as tf

# Set the seed for NumPy
np.random.seed(42)

# Set the seed for TensorFlow
tf.random.set_seed(42)

import pandas as pd
import os, glob
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import PIL
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array, array_to_img
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import to_categorical
tf.__version__

# from tensorflow.keras.preprocessing.image import load_img, img_to_array



In [ ]:
# Custom functions:
%load_ext autoreload
%autoreload 2
# sys.path.append(os.path.abspath("../../"))
import ann_functions as af

In [ ]:
# Checking the contents of data folder
data_dir = "./American Sign Language Letters.v1-v1.multiclass/"
data_dir

In [ ]:
# Getting list of img file paths (ONLY, did not make recursuve so no folders)
img_files = glob.glob(data_dir+"**/*")#, recursive=True)
len(img_files)

In [ ]:
# Preview an example image (at full size)
img_loaded = load_img(img_files[0])
img_data = img_to_array(img_loaded)
img_data.shape

### 🎛️ Project Params

In [ ]:
## Set project-wide parameters
# # Saving image params as vars for reuse
BATCH_SIZE = 32
IMG_HEIGHT = 128
IMG_WIDTH = 128
TRAIN_SPLIT = 0.7  # Proportion of data for training
VAL_SPLIT = 0.15  # Proportion of data for validation (remaining will be for test)

## Data

### Prepare CSV of Filenames + Labels

In [ ]:

# Load the CSV file
csv_path = os.path.join(data_dir,"train","_classes.csv")
df = pd.read_csv(csv_path)

df = df.convert_dtypes()
# df['filename'] = df['filename']
df = df.set_index('filename')
df = df.astype(float)
df

In [ ]:
# Combine label columns into single column
df.loc[:,'label'] = df.apply(lambda x: x.idxmax(), axis=1)
display(df.head(2))
df['label'].value_counts(1).sort_index()

In [ ]:
n_examples = df.drop(columns=['filename', 'label'], errors='ignore').sum()
n_examples

In [ ]:
label_cols = sorted(df.drop(columns=['filename','label'], errors='ignore').columns)
label_lookup = {i:label for i,label in enumerate(label_cols)}
label_lookup

In [ ]:
## Get the filepaths and labels
df = df.reset_index(drop=False)

df['filepath'] = df.loc[:,'filename'].astype(str).map(lambda x: os.path.join(data_dir, "train/", x)).values
filepaths = df['filepath']

labels = df[label_cols].astype(float).values
filepaths[0], labels[0]

In [ ]:
files_exist = np.array([os.path.exists(f) for f in filepaths])
files_exist.all()

In [ ]:
# labels

In [ ]:
display(load_img(filepaths[0]))
print(f"Letter: {label_lookup[np.argmax(labels[0])]}")

In [ ]:

# # Create a TensorFlow dataset from the image paths and labels
# def load_image(image_path, label, img_height=128, img_width=128):
#     target_size=(img_height, img_width)
#     image = load_img(image_path, target_size=target_size)
#     image = img_to_array(image)
#     image = image / 255.0  # Normalize the image
#     return image, label


# Function to load and preprocess images
def load_image(filename, label, img_height=128, img_width=128):
    img = tf.io.read_file(filename)
    # img = tf.image.decode_image(img, channels=3)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img.set_shape([None, None, 3])  # Explicitly set the shape
    img = tf.image.resize(img, [img_height, img_width])
    # img = img / 255.0  # Normalize the image
    return img, label


In [ ]:
ex_img, ex_label = load_image(filepaths[0], labels[0])
print(ex_img.shape)
display(array_to_img(ex_img))
print(f"Label: {ex_label}")
print(f"Label: {label_lookup[np.argmax(ex_label)]}")

### EDA

In [ ]:
eda_df = df[['filepath', 'label']]
eda_df

In [ ]:
# ## Showing example of each letter
# label_lookup.values()

In [ ]:
# labels = sorted(df['label'].unique())
# labels

In [ ]:
import seaborn as sns
# sns.set_theme(style="whitegrid")

fig, ax = plt.subplots(figsize=(10, 6))
ax = sns.countplot(data=eda_df, x='label',order=label_lookup.values(),
              hue='label', dodge=False,palette=sns.color_palette("icefire",n_colors=len(label_lookup)),
              ax=ax)
ax.set(title="Distribution of Labels", xlabel="Letter", ylabel="Count")

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
fig.savefig("images/label_dist.png", dpi=300, bbox_inches='tight', transparent=False)

#### Display Example of Each

In [ ]:
### Plot example of each letter
import os
os.makedirs("images", exist_ok=True)


ncols = 6
unique_labels = sorted(eda_df['label'].unique())
nrows = len(unique_labels)//ncols + 1

fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15,15))
axes = axes.flatten()


for i, label in enumerate(unique_labels):
    fpath = eda_df.loc[ eda_df['label']==label,'filepath'].sample(1).values[0]
    
    loaded = plt.imread(fpath)
    axes[i].imshow(loaded)
    axes[i].set_title(label)
    axes[i].axis('off')
    

# remove unused axes
axes_labels_diff =  len(axes) - len(unique_labels)

if axes_labels_diff>0:
    for ax in axes[-axes_labels_diff:]:
        
        # difference = len(axes)
        fig.delaxes(ax=ax)   
        
fig.tight_layout()

fig.savefig("images/eda_example_letters.png", dpi=300, bbox_inches='tight', transparent=False)

    
    
    
    

In [ ]:
# ### Plot example of each letter
# import os
# os.makedirs("images", exist_ok=True)

# ncols = 6
# unique_labels = sorted(eda_df['label'].unique())
# nrows = len(unique_labels)//ncols + 1

# fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15,15))
# axes = axes.flatten()


# for i, label in enumerate(unique_labels):
#     fpath = eda_df.loc[ eda_df['label']==label,'filepath'].sample(1).values[0]
    
#     loaded = plt.imread(fpath)
#     axes[i].imshow(loaded)
#     axes[i].set_title(label)
#     axes[i].axis('off')
    

# # remove unused axes
# axes_labels_diff =  len(axes) - len(unique_labels)

# if axes_labels_diff>0:
#     for ax in axes[-axes_labels_diff:]:
        
#         # difference = len(axes)
#         fig.delaxes(ax=ax)   
        
# fig.tight_layout()

# fig.savefig("images/eda_example_letters.png", dpi=300, bbox_inches='tight', transparent=False)

    
    
    
    

In [ ]:
# df.loc[:,'label'] = df.loc[:,label_cols].apply(lambda x: x.idxmax(), axis=1)
# df['label'].value_counts().sort_index()

In [ ]:
# image_paths = np.array(image_paths)
# labels = np.array(labels)
# image_paths.shape, labels.shape

### Construct Train/Test/Val Tensorflow Datasets

In [ ]:
# load_image(image_paths[0], labels[0])
dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))

# Shuffle and batch the dataset
dataset = dataset.shuffle(buffer_size=len(dataset), reshuffle_each_iteration=False)

dataset.take(1).get_single_element()

In [ ]:
tf.data.experimental.AUTOTUNE

In [ ]:

## Map the load_image function to the dataseta
dataset = dataset.map(lambda x,y: load_image(x,y), num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset.take(1).get_single_element()

In [ ]:
# Determine split sizes
total_size = len(dataset)
train_size = int(TRAIN_SPLIT * total_size)
val_size = int(VAL_SPLIT * total_size)
test_size = total_size - train_size - val_size
print(f"{train_size=}, {test_size=}, {val_size=}")


In [ ]:
# Split the dataset
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)
test_dataset = dataset.skip(train_size + val_size)

# # Cache the datset for faster access
# train_dataset = train_dataset.cache()
# val_dataset = val_dataset.cache()
# test_dataset = test_dataset.cache() 


In [ ]:
# Batch and prefetch the datasets
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Shuffle the trainin data
train_dataset = train_dataset.shuffle(buffer_size=train_dataset.cardinality(), 
                                      reshuffle_each_iteration=True) # DOUBLE CHECK BATCH_SIZE * 8


In [ ]:

# Use the datasets
for images, labels in train_dataset.take(1):
    print(f"Train batch - images: {images.shape}, labels: {labels.shape}")
    
for images, labels in val_dataset.take(1):
    print(f"Val batch - images: {images.shape}, labels: {labels.shape}")
    
    
for images, labels in test_dataset.take(1):
    print(f"Test batch - images: {images.shape}, labels: {labels.shape}")
    

### Baseline Model (From towardsdatascience blog)
- from https://towardsdatascience.com/sign-language-recognition-with-advanced-computer-vision-7b74f20f3442


In [ ]:
# Moedl from https://towardsdatascience.com/sign-language-recognition-with-advanced-computer-vision-7b74f20f3442
# from tensorflow
def make_model(name='towards-data-science',show_summary=False, use_schedule=False):
    model = models.Sequential(name=name)
    model.add(layers.Rescaling(1./255 , input_shape = (IMG_HEIGHT,IMG_WIDTH,3)))
    
    model.add(layers.Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' ))#, input_shape = (28,28,1)))
    
    model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    model.add(layers.Conv2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    model.add(layers.Conv2D(25 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
    
    model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    
    # Final layers
    model.add(layers.Flatten())
    model.add(layers.Dense(units = 512 , activation = 'relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(units = len(label_lookup   ) , activation = 'softmax'))
    
    
    ## JMI:
    if use_schedule:
        lr_schedule = optimizers.schedules.ExponentialDecay(
            initial_learning_rate=0.01, decay_steps=10000, decay_rate=0.95
        )  # 0.9)
        optimizer = optimizers.legacy.Adam(learning_rate=lr_schedule)
    else:
        optimizer = optimizers.legacy.Adam()#learning_rate=0.01)
        
    model.compile(optimizer=optimizer, 
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])
    # model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
    if show_summary:
        model.summary()
    return model


# Demonstrate model architecture
model = make_model(show_summary=True)

#### `def get_callbacks`

In [ ]:


def get_callbacks(monitor='val_accuracy', patience=10, start_from_epoch=5, restore_best_weights=False):
    """
    Returns a list of callbacks for training a model.

    Parameters:
    - monitor (str): The metric to monitor. Default is 'val_accuracy'.
    - patience (int): The number of epochs with no improvement after which training will be stopped. Default is 15.
    - start_from_epoch (int): The epoch from which to start counting the patience. Default is 3.
    - restore_best_weights (bool): Whether to restore the weights of the best epoch. Default is False.

    Returns:
    - callbacks (list): A list of callbacks to be used during model training.
    """
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=patience,start_from_epoch=start_from_epoch,
                                                      monitor=monitor,
                                                      restore_best_weights=restore_best_weights, verbose=1)
    return [early_stopping]




In [ ]:
## Scratch Code to create function
# # Baseline model
# model = make_model(show_summary=False, use_schedule=False)
# history = model.fit(train_dataset,epochs = 100,#0 ,
#                     validation_data = val_dataset, callbacks=get_callbacks())
# af.evaluate_classification_network(model,X_test=test_dataset,history=history, figsize=(15,15),
#                                    target_names=label_lookup.values());

## Updating Evaluation to Handle Large # Classes

> With 26 classes, it is difficult to scan the performance for each class visually. Adding code to convert results to a datafarme and use pandas styling to visualize

In [ ]:
## Scratch Code to create function
# results_dict = af.evaluate_classification_network(model,X_test=test_dataset,history=history, figsize=(15,15), output_dict=True, target_names=label_lookup.values())
# results_dict.keys()

In [ ]:
## Scratch Code to create function
# results_dict['test'].keys()

In [ ]:
## Scratch Code to create function
# from IPython.display import clear_output
# results_dict = af.evaluate_classification_network(model,X_test=test_dataset,history=history, figsize=(15,15), output_dict=True, target_names=label_lookup.values())
# clear_output()
# results = results_dict['test'].copy()
# try:
#     accuracy = results.pop('accuracy')
#     macro_avg = results.pop('macro avg')
#     _ = results.pop('weighted avg')
# except Exception as e:
#     display(e)

# results_df = pd.DataFrame(results).T
# results_df['support'] = results_df['support'].astype(int)
# results_df

In [ ]:
## Scratch Code to create function

# overall_results = pd.DataFrame(macro_avg, index=['macro avg'])#.T
# overall_results.insert(0,'accuracy',accuracy)
# overall_results

In [ ]:
## Scratch Code to create function

# accuracy, macro_avg
# results_df

In [ ]:
def get_results_df(results_dict, results_key='test', 
                   average_rowname= 'macro avg',
                   include_support = True,
                   include_macro_avg=True):
    """
    Convert a results dictionary into a pandas DataFrame.

    Parameters:
    - results_dict (dict): A dictionary containing the results.
    - results_key (str): The key in the dictionary that contains the results. Default is 'test'.
    - average_rowname (str): The name of the row that represents the average. Default is 'macro avg'.
    - include_support (bool): Whether to include the 'support' column in the DataFrame. Default is True.
    - include_macro_avg (bool): Whether to include the 'macro avg' row in the DataFrame. Default is True.

    Returns:
    - results_df (pandas DataFrame): A DataFrame containing the results.

    """
    results = results_dict[results_key].copy()
    
    # Remove accuracy and macro avg from results
    accuracy = results.pop('accuracy')
    macro_avg = results.pop('macro avg')
    _ = results.pop('weighted avg')
    
    # Create DataFrames
    results_df = pd.DataFrame(results).T
    
    if include_macro_avg:
        overall_results = pd.DataFrame(macro_avg, index=[average_rowname])#.T
    
        ## Concatenate the overall results to the results_df
        results_df = pd.concat([results_df, overall_results],axis=0)
        results_df.loc[average_rowname,'accuracy'] = accuracy
    
    # Recast support as int
    results_df['support'] = results_df['support'].astype(int)

    # Move the support column to the end
    # results_df = results_df[ results_df.drop(columns='support').columns.tolist() + ['support']]
    
    if not include_support:
        results_df = results_df.drop(columns='support')
    
    return results_df


In [ ]:
## Scratch Code to create function

# results_dict = af.evaluate_classification_network(model,X_test=test_dataset,history=history, figsize=(15,15), output_dict=True, target_names=label_lookup.values())
# results_test = get_results_df(results_dict)
# results_test


In [ ]:
## Scratch Code to create function

# results_test = get_results_df(results_dict, include_macro_avg=False, include_support=False)
# results_test.style.bar(color='#5fba7d')

In [ ]:
## Scratch Code to create function

# results_test.style.background_gradient(cmap='Greens', vmax=1,vmin=0, axis=0)

### Added new kwarg to evaluate_classification_network

In [ ]:
## Scratch Code to create function

# results_dict = af.evaluate_classification_network(model,X_test=test_dataset,history=history, figsize=(15,15),
#                                                   output_dict=True, target_names=label_lookup.values(),
#                                                   as_frame=True,
#                                                   frame_include_macro_avg=False,frame_include_support=False)
# # results_test = get_results_df(results_dict)
# # results_test
# results_dict['test'].style.bar(color='#5fba7d')


### Updated custom Evaluation Function (for Notebook use only)

In [ ]:
def custom_evaluate_classification_network(model, X_test, history=None, figsize=(15,15), target_names=None,
                                             as_frame=True, frame_include_macro_avg=False, frame_include_support=False,
                                             display_bar=True):
    """
    Evaluate a classification model on a test dataset.

    Parameters:
    - model: The trained classification model.
    - X_test: The test dataset.
    - history: The training history of the model (optional).
    - figsize: The size of the figure for plotting the evaluation results (default: (15, 15)).
    - target_names: The names of the target classes (default: None).
    - as_frame: Whether to return the evaluation results as a pandas DataFrame (default: True).
    - frame_include_macro_avg: Whether to include macro average metrics in the DataFrame (default: False).
    - frame_include_support: Whether to include support values in the DataFrame (default: False).
    - display_bar: Whether to display the evaluation results as a styled bar chart (default: True).

    Returns:
    - results_dict: A dictionary containing the evaluation results.
    """
    if target_names is None:
        # label_lookup is in the global scope
        target_names = label_lookup.values()
        
    results_dict = af.evaluate_classification_network(model,
                                                      X_test=X_test,history=history, figsize=figsize,
                                                  output_dict=True, target_names=target_names,#label_lookup.values(),
                                                  as_frame=True,
                                                    frame_include_macro_avg=frame_include_macro_avg,
                                                    frame_include_support=frame_include_support)
    # results_test = get_results_df(results_dict)
    # results_test
    if display_bar:
        display(results_dict['test'].style.bar(color='#5fba7d'))
    return results_dict

In [ ]:
# results_dict['test'].style.bar(color='#5fba7d')

## Model 1 (with New Function)

In [ ]:
## Show model architecture
model = make_model(show_summary=True, use_schedule=False)


In [ ]:
%%time
# Baseline model
model = make_model(show_summary=False, use_schedule=False)
history = model.fit(train_dataset,epochs = 100 ,validation_data = val_dataset, callbacks=get_callbacks())
# results_dict = af.evaluate_classification_network(model,X_test=test_dataset,history=history, figsize=(15,15),
#                                                   output_dict=True, target_names=label_lookup.values(),
#                                                   as_frame=True,
#                                                   frame_include_macro_avg=False,frame_include_support=False)
results = custom_evaluate_classification_network(model,X_test=test_dataset,history=history, figsize=(15,15),
                                                 target_names=label_lookup.values(),display_bar=True);

### Model 1-LR: Adding LR Scheduling to Model 1

In [ ]:
%%time
# Baseline model
model = make_model(show_summary=False, 
                   use_schedule=True # Adding learning rate scheduling
                   )
history = model.fit(train_dataset,epochs = 100 ,validation_data = val_dataset, callbacks=get_callbacks())

results = custom_evaluate_classification_network(model,X_test=test_dataset,history=history, figsize=(15,15),
                                                 target_names=label_lookup.values(),display_bar=True);


## Model2 (custom from scratch)

In [ ]:

def make_model2(name='CNN1',show_summary=False):
    
    model = models.Sequential(name=name)
    # Using rescaling layer to scale pixel values
    model.add(layers.Rescaling(1./255 , input_shape = (IMG_HEIGHT,IMG_WIDTH,3)))
    
    # Convolutional layer
    model.add(
        layers.Conv2D(
            filters=16,  # How many filters you want to use
            kernel_size=3, # size of each filter
            # input_shape=input_shape,
            padding='same')) 
    # Pooling layer
    model.add(layers.MaxPooling2D(pool_size=2))  # Size of pooling


    # Convolutional layer
    model.add(
        layers.Conv2D(
            filters=32,#64,  # How many filters you want to use
            kernel_size=3,  # size of each filter
            # input_shape=input_shape,
            padding='same')) 
    # Pooling layer
    model.add(layers.MaxPooling2D(pool_size=2))  # Size of pooling
    
    # Flattening layer
    model.add(layers.Flatten())
    # Output layer
    model.add(
        layers.Dense(len(label_lookup), activation="softmax") )  
    ## Adding learning rate decay
    lr_schedule = optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.01, decay_steps=10000, decay_rate=0.95
    )  # 0.9)
    optimizer = optimizers.legacy.Adam(learning_rate=lr_schedule)
    
    model.compile(optimizer=optimizer, 
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])
    if show_summary:
        model.summary()
    return model

In [ ]:
## Show model architecture
model2 = make_model2(show_summary=True)


In [ ]:
%%time
model2 = make_model2()
history2 = model2.fit(train_dataset,epochs = 100 ,validation_data = val_dataset, callbacks=get_callbacks())
# results_dict = af.evaluate_classification_network(model2,X_test=test_dataset,history=history2, figsize=(15,15), output_dict=True, target_names=label_lookup.values())
# results_dict.keys()
results_dict = custom_evaluate_classification_network(model2,X_test=test_dataset,history=history2, figsize=(20,20), 
                                                      target_names=label_lookup.values(),
                                                      as_frame=True, frame_include_macro_avg=False, frame_include_support=False,
                                                      display_bar=True)

In [ ]:
1/26

## Transfer Learning



| Model             |   Size (MB) | Top-1 Accuracy   | Top-5 Accuracy   | Parameters   | Depth   | Time (ms) per inference step (CPU)   | Time (ms) per inference step (GPU)   |
|:------------------|------------:|:-----------------|:-----------------|:-------------|:--------|:-------------------------------------|:-------------------------------------|
| **VGG16**             |      528    | 71.3%            | 90.1%            | 138.4M       | 16      | 69.5                                 | 4.2                                  |
| **EfficientNetB0**    |       29    | 77.1%            | 93.3%            | 5.3M         | 132     | 46.0                                 | 4.9                                  |
| **InceptionV3**       |       92    | 77.9%            | 93.7%            | 23.9M        | 189     | 42.2                                 | 6.9                                  |

*Excerpt from Source: "https://keras.io/api/applications/"*

In [ ]:
input_shape = (IMG_HEIGHT,IMG_WIDTH,3)
input_shape

### VGG16

In [ ]:
# Downloading just the convolutional base
vgg16_base = tf.keras.applications.VGG16(
    include_top=False, weights="imagenet", input_shape=input_shape
)
# Prevent layers from base_model from changing 
vgg16_base.trainable = False

# Create the preprocessing lamdba layer
# Create a lambda layer for the preprocess input function for the model
lambda_layer_vgg16 = tf.keras.layers.Lambda(
    tf.keras.applications.vgg16.preprocess_input, name="preprocess_input"
)



def make_vgg16_model(show_summary=False):
    model = models.Sequential(name="VGG16")
    # Use input layer (lambda layer will handle rescaling).
    model.add(tf.keras.layers.Input(shape=input_shape))

    ## Adding preprocessing lamabda layer
    model.add(lambda_layer_vgg16)

    # Add pretrained base
    model.add(vgg16_base)

    # Flattening layer
    model.add(layers.Flatten())

    ## Adding a Hidden Dense Layer
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dropout(0.5))

    # Output layer
    model.add(layers.Dense(len(label_lookup.values()), activation="softmax"))

    model.compile(
        optimizer=tf.keras.optimizers.legacy.Adam(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=["accuracy"],
    )
    
    if show_summary:
        model.summary()
        
    return model


In [ ]:
%%time
# Baseline model
model = make_vgg16_model(show_summary=False, 
                   use_schedule=True # Adding learning rate scheduling
                   )
history = model.fit(train_dataset,epochs = 100 ,validation_data = val_dataset, callbacks=get_callbacks())

results = custom_evaluate_classification_network(model,X_test=test_dataset,history=history, figsize=(15,15),
                                                 target_names=label_lookup.values(),display_bar=True);


In [ ]:
raise Exception('not ready for below')

In [ ]:
# Download EfficientNet base
efficientnet_base =tf.keras.applications.EfficientNetB0(include_top=False, 
                                                       input_shape=input_shape)

# Make it not-trainable
efficientnet_base.trainable=False

# add preprocessing lambda layer
lambda_layer_efficient = tf.keras.layers.Lambda(tf.keras.applications.efficientnet.preprocess_input, 
                                      name='preprocess_input_enet')

def build_efficientnet_model():
    model = models.Sequential(name="EfficientNetB0")
    # Use input layer (lambda layer will handle rescaling).
    model.add(tf.keras.layers.Input(shape=input_shape))

    ## Adding preprocessing lamabda layer
    model.add(lambda_layer_efficient)

    # Add pretrained base
    model.add(efficientnet_base)

    # Flattening layer
    model.add(layers.Flatten())

    ## Adding a Hidden Dense Layer
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dropout(0.5))

    # Output layer
    model.add(layers.Dense(len(label_lookup.values()), activation="softmax"))

    model.compile(
        optimizer=tf.keras.optimizers.legacy.Adam(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=["accuracy"],
    )
    model.summary()
    return model

# vk.layered_view(efficientnet_base, legend=True)

## To Do: Keras Tuner

In [ ]:
# ## Fit and evaluate model with custom function
# model2 = make_model2()
# history2 = model2.fit(train_dataset,epochs = 100 ,validation_data = val_dataset, callbacks=get_callbacks())
# results_dict = custom_evaluate_classification_network(model2,X_test=test_dataset,history=history2, figsize=(15,15), 
#                                                       target_names=label_lookup.values(),
#                                                       as_frame=True, frame_include_macro_avg=False, frame_include_support=False,
#                                                       display_bar=True)

## To Do: Add LimeExplanations

In [ ]:
BEST_MODEL = None

#### Convert test data to numpy arrays

In [ ]:
%%time
# timing WITH converting classes
y_test, y_hat_test, X_test = af.get_true_pred_labels_images(BEST_MODEL,test_dataset,
                                                         convert_y_for_sklearn=True)
y_test[0], y_hat_test[0]

In [ ]:
# select an image index to use/view
i = 10

# Show actual-sized image with keras
display(array_to_img(X_test[i]))
print(f"True Label: {label_lookup[y_test[i]]}")
print(f"Predicted: {label_lookup[y_hat_test[i]]}")

### LimeExplainer

In [ ]:
from skimage.segmentation import mark_boundaries
from lime import lime_image

In [ ]:
explainer = lime_image.LimeImageExplainer(verbose=False)#,random_state=321)
explainer

In [ ]:
# Get the explanation object for the chosen
explanation = explainer.explain_instance(X_test[i], # Convert image values to ints    
                                         model.predict, # Prediction method/function
                                         top_labels=1, # How many of the labels to explain [?]
                                         hide_color=0, #
                                         num_samples=1000,
                                        )

In [ ]:
# Stored original image
plt.imshow(explanation.image)#.astype(int));
plt.axis('off');

In [ ]:
# Explanation split image into "segments"
plt.imshow(explanation.segments); 

In [ ]:
# Unique Segments
np.unique(explanation.segments)

In [ ]:
#pros and cons
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], 
                                            positive_only=True, 
                                            num_features=5, 
                                            hide_rest=True)
plt.imshow(mark_boundaries(temp, mask))
plt.axis('off')
plt.title('Segments that Positively Pushed Prediction');

In [ ]:
#pros and cons
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], 
                                            negative_only=True, 
                                            positive_only=False,
                                            num_features=5, 
                                            hide_rest=True)

#pros and cons
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], 
                                            negative_only=False, 
                                            positive_only=False,
                                            num_features=5, 
                                            hide_rest=False)
plt.imshow(mark_boundaries(temp, mask))
plt.axis('off')
plt.title(f'Segments that Pushed Prediction Towards (Green) or Away (Red) from {label}');

In [ ]:
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries

from IPython.display import clear_output

def plot_comparison(main_image, img, mask):
    """Adapted from Source:
    https://coderzcolumn.com/tutorials/artificial-intelligence/lime-explain-keras-image-classification-network-predictions"""
    fig,axes = plt.subplots(ncols=4,figsize=(15,5))

    # show original image
    ax = axes[0]
    ax.imshow(main_image)#.astype(int))#, cmap="gray");
    ax.set_title("Original Image")
    ax.axis('off')

    ax =axes[1]
    ax.imshow(img)#.astype(int));
    ax.set_title("Image")
    ax.axis('off')
    
    ax = axes[2]
    ax.imshow(mask);
    ax.set_title("Mask")
    ax.axis('off')
    
    ax = axes[3]
    ax.imshow(mark_boundaries(img,
                              mask, color=(0,1,0)));
    ax.set_title("Image+Mask Combined");
    ax.axis('off')
    fig.tight_layout()
    

In [ ]:
plot_comparison(X_test[i], temp, mask)

### Explaining an Incorrect Prediction